<a href="https://colab.research.google.com/github/naratiz/Precoce-MS/blob/main/4_Modeling_Classifica%C3%A7%C3%A3o/Otimiza%C3%A7%C3%A3o_Classifica%C3%A7%C3%A3o_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install optuna teste

In [ ]:
import pandas as pd
import optuna
import numpy as np
import seaborn as sns
import time
import matplotlib.pyplot as plt
import concurrent.futures
import itertools
import logging
import sys

from sklearn.model_selection import cross_val_score,train_test_split, StratifiedKFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from imblearn.combine import SMOTEENN 

In [ ]:
import os 
n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

Number of CPUs in the system: 8


# Base de Dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Dissertação/dados/procucao_gado_MS_train.csv"
# path = 'dados/procucao_gado_MS.csv'
df = pd.read_csv(path, sep=';')

In [ ]:
print('A base de dados apresenta {} registros e {} características.'.format(df.shape[0], df.shape[1]))

A base de dados apresenta 768751 registros e 48 características.


### Separando as características preditoras do atributo alvo

In [ ]:
SEED = 42

In [ ]:
y = df['categoria']
# Retirar a variável alvo e as variáveis obtidas após o abate 
X = df.drop(['categoria'],axis=1)

In [ ]:
# Para remover UserWarning do XGBClassifier: Use o parêmetro use_label_encoder=False 
# e codifique as classes no inervalo[0 ... n]
y=y.replace(6, 0)

### StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## PCA

In [ ]:
pca = PCA(n_components=30)
pca.fit(X_scaled)
X_scaled_pca = pca.transform(X_scaled)

## Baseline

In [ ]:
skfolds = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)

In [ ]:
xgb_baseline = XGBClassifier(objective="multi:softmax", num_class= 6, eval_metric='mlogloss', random_state=SEED, nthread=-1,  use_label_encoder=False)
t0 = time.time()
score = cross_val_score(xgb_baseline, X_scaled, y, cv = skfolds)
print('Acurácia média da validação cruzada : %0.3f"' % score.mean())
print("Pronto em:  %0.3f" %(time.time()-t0))

Acurácia média da validação cruzada : 0.400"
Pronto em:  1195.628


## Otimização dos hiperparâmetros

In [ ]:
skfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "xgb-study"  # Unique identifier of the study.
storage_name = "sqlite:///sample_data/{}.db".format(study_name)

In [ ]:
# função de otimização baseada em uma função objetivo
def tune (objective):
    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="maximize")
    study.optimize(objective, timeout=61200, n_trials=100)

    params = study.best_params
    print(f"Trial: {study.best_trial.number}")
    params = study.best_params
    print("\n")
    print(f"Melhor seleção de parâmetros : \n{params}")
    print("\n")
    print(f"Acurácia : {study.best_value:.3f}")
    
    return study, params

# XGBClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=SEED)

In [ ]:
def xgb_objective(trial: optuna.Trial): 
    xgb_params = {
        'nthread' : -1,
        'objective':'multi:softmax',
        'num_class': 6,
        'eval_metric': 'mlogloss',
        'random_state': SEED,       
        'gamma' : trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 5),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 10.0),
        'max_depth': trial.suggest_int('max_depth', 2, 30),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.2, 0.9), 
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'eta': trial.suggest_categorical('eta', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02,0.5,1.0]),
    }
    model = XGBClassifier(**xgb_params, use_label_encoder=False) 
    scores = cross_val_score(model, X_train, y_train, n_jobs=-1, cv=skfolds)
    return scores.mean()

In [ ]:
t0 = time.time()
study, best_params = tune(xgb_objective)  
print("Pronto em:  %0.3f" %(time.time()-t0))

[I 2022-03-21 23:48:22,909] Using an existing study with name 'xgb-study' instead of creating a new one.


Using an existing study with name 'xgb-study' instead of creating a new one.
Using an existing study with name 'xgb-study' instead of creating a new one.


[I 2022-03-22 00:26:36,086] Trial 97 finished with value: 0.5111340301974449 and parameters: {'gamma': 6.945081212306528e-07, 'reg_alpha': 2.140366923972721e-06, 'lambda': 1.584839227670745e-07, 'max_depth': 21, 'min_child_weight': 6, 'colsample_bytree': 0.28564370992110033, 'subsample': 1.0, 'eta': 0.02}. Best is trial 76 with value: 0.5111433217189314.


Trial 97 finished with value: 0.5111340301974449 and parameters: {'gamma': 6.945081212306528e-07, 'reg_alpha': 2.140366923972721e-06, 'lambda': 1.584839227670745e-07, 'max_depth': 21, 'min_child_weight': 6, 'colsample_bytree': 0.28564370992110033, 'subsample': 1.0, 'eta': 0.02}. Best is trial 76 with value: 0.5111433217189314.
Trial 97 finished with value: 0.5111340301974449 and parameters: {'gamma': 6.945081212306528e-07, 'reg_alpha': 2.140366923972721e-06, 'lambda': 1.584839227670745e-07, 'max_depth': 21, 'min_child_weight': 6, 'colsample_bytree': 0.28564370992110033, 'subsample': 1.0, 'eta': 0.02}. Best is trial 76 with value: 0.5111433217189314.


[I 2022-03-22 01:05:54,193] Trial 98 finished with value: 0.5111934959349592 and parameters: {'gamma': 7.904470077101956e-07, 'reg_alpha': 1.892577558598394e-06, 'lambda': 1.6440490656916741e-07, 'max_depth': 21, 'min_child_weight': 5, 'colsample_bytree': 0.2937657729656727, 'subsample': 1.0, 'eta': 0.02}. Best is trial 98 with value: 0.5111934959349592.


...

Trial 125 finished with value: 0.5113365853658537 and parameters: {'gamma': 1.8030627974482705e-05, 'reg_alpha': 8.178450742773128e-07, 'lambda': 1.0479107603833961e-07, 'max_depth': 17, 'min_child_weight': 17, 'colsample_bytree': 0.32082495952188383, 'subsample': 1.0, 'eta': 0.02}. Best is trial 125 with value: 0.5113365853658537.
Trial 125 finished with value: 0.5113365853658537 and parameters: {'gamma': 1.8030627974482705e-05, 'reg_alpha': 8.178450742773128e-07, 'lambda': 1.0479107603833961e-07, 'max_depth': 17, 'min_child_weight': 17, 'colsample_bytree': 0.32082495952188383, 'subsample': 1.0, 'eta': 0.02}. Best is trial 125 with value: 0.5113365853658537.


[I 2022-03-22 16:23:58,897] Trial 126 finished with value: 0.5110652729384436 and parameters: {'gamma': 3.8173696931783926e-05, 'reg_alpha': 4.865283850550834e-08, 'lambda': 3.029101807357798e-08, 'max_depth': 20, 'min_child_weight': 1, 'colsample_bytree': 0.3246432566474443, 'subsample': 1.0, 'eta': 0.02}. Best is trial 125 with value: 0.5113365853658537.


Trial 126 finished with value: 0.5110652729384436 and parameters: {'gamma': 3.8173696931783926e-05, 'reg_alpha': 4.865283850550834e-08, 'lambda': 3.029101807357798e-08, 'max_depth': 20, 'min_child_weight': 1, 'colsample_bytree': 0.3246432566474443, 'subsample': 1.0, 'eta': 0.02}. Best is trial 125 with value: 0.5113365853658537.
Trial 126 finished with value: 0.5110652729384436 and parameters: {'gamma': 3.8173696931783926e-05, 'reg_alpha': 4.865283850550834e-08, 'lambda': 3.029101807357798e-08, 'max_depth': 20, 'min_child_weight': 1, 'colsample_bytree': 0.3246432566474443, 'subsample': 1.0, 'eta': 0.02}. Best is trial 125 with value: 0.5113365853658537.


[I 2022-03-22 16:48:58,203] Trial 127 finished with value: 0.5105672473867596 and parameters: {'gamma': 1.2862597687136438e-05, 'reg_alpha': 8.999971138110206e-07, 'lambda': 3.668175651975311e-07, 'max_depth': 17, 'min_child_weight': 22, 'colsample_bytree': 0.22845922944337452, 'subsample': 1.0, 'eta': 0.01}. Best is trial 125 with value: 0.5113365853658537.


Trial 127 finished with value: 0.5105672473867596 and parameters: {'gamma': 1.2862597687136438e-05, 'reg_alpha': 8.999971138110206e-07, 'lambda': 3.668175651975311e-07, 'max_depth': 17, 'min_child_weight': 22, 'colsample_bytree': 0.22845922944337452, 'subsample': 1.0, 'eta': 0.01}. Best is trial 125 with value: 0.5113365853658537.
Trial 127 finished with value: 0.5105672473867596 and parameters: {'gamma': 1.2862597687136438e-05, 'reg_alpha': 8.999971138110206e-07, 'lambda': 3.668175651975311e-07, 'max_depth': 17, 'min_child_weight': 22, 'colsample_bytree': 0.22845922944337452, 'subsample': 1.0, 'eta': 0.01}. Best is trial 125 with value: 0.5113365853658537.
Trial: 125


Melhor seleção de parâmetros : 
{'colsample_bytree': 0.32082495952188383, 'eta': 0.02, 'gamma': 1.8030627974482705e-05, 'lambda': 1.0479107603833961e-07, 'max_depth': 17, 'min_child_weight': 17, 'reg_alpha': 8.178450742773128e-07, 'subsample': 1.0}


Acurácia : 0.511
Pronto em:  61235.416


In [ ]:
# Determina quais parâmetros tiveram o impacto mais significativo no desempenho geral do modelo.
optuna.visualization.plot_param_importances(study)